In [1]:
from werpy_weighted import werpy
import whisper
from whisper.tokenizer import get_tokenizer

In [2]:
%cd "D:\Schoolwork\TERM 3\WORK\visual_prosody"

D:\Schoolwork\TERM 3\WORK\visual_prosody


In [3]:
import os, sys, glob, shutil
import argparse
import json
import yaml
import numpy as np
from pprint import pprint
import logging
import os
import os.path as op

import torch
import torch.nn.functional as F
import numpy as np
import torchaudio
import pandas as pd
from tqdm.notebook import tqdm

# v4

In [4]:
tokenizer = get_tokenizer(multilingual=False)  # use multilingual=True if using multilingual model
number_tokens = [
    i 
    for i in range(tokenizer.eot)
    if all(c in "0123456789" for c in tokenizer.decode([i]).removeprefix(" "))
]

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

whisper_model = whisper.load_model('base.en').to(device)

In [6]:
split_txt_val_path = r'.\preprocessed_data\Ego4D_final_v4\val.txt'
val_uids = []
gt_transcripts_dict = {}
with open(split_txt_val_path) as file:
    for line in file:
        # print(line.split('|')[0])
        val_uids.append(line.split('|')[0])
        gt_transcripts_dict[line.split('|')[0]] = line.split('|')[-1]

In [7]:
len(val_uids)

2641

In [8]:
gt_folder = r'.\raw_data\Ego4D_final_v4\val\Ego4D_final_v4'

In [9]:
%%time
uids = []
whisper_transcripts = []
gt_transcripts = []

for uid in tqdm(val_uids):
    uids.append(uid)
    output = whisper_model.transcribe(
            op.join(gt_folder, f"{uid}.wav"),
            suppress_tokens=[-1] + number_tokens,
    )
    whisper_transcript = output["text"]
    gt_transcript = gt_transcripts_dict[uid]

    whisper_transcripts.append(whisper_transcript)
    gt_transcripts.append(gt_transcript)


  0%|          | 0/2641 [00:00<?, ?it/s]

CPU times: total: 58min 20s
Wall time: 26min 44s


In [11]:
whisper_df_v4gt = pd.DataFrame({
    'uid': uids,
    'whisper_transcripts': whisper_transcripts,
    'gt_transcripts': gt_transcripts,
})

In [12]:
whisper_df_v4gt.head()

,uid,whisper_transcripts,gt_transcripts
0,5c7783ef-05ba-4b35-8a91-360283d536dd,It depends on me. If I call you liar or I cha...,it depends on me if i call you liar or i chal...
1,d0d1d7a2-0a96-45e5-a0e4-75a59055da80,I'm gonna paste my stuff.,i gotta pace myself \n
2,bd4f326f-29b9-4c9d-9587-d1d7715f9ac5,"Oh, I'm not moving. You see me moving?",oh i'm not moving you see me moving \n
3,a4033601-48cc-4b05-a81b-2475bfe5a64f,Anything towards the back with seams is less ...,anything towards the back will seem is less l...
4,a94a6ca4-fb80-42ce-a452-14e850cc95b6,"Yes, I wish you none.",you got solution yet \n


In [13]:
whisper_df_v4gt.to_csv(r".\jupyter_walkthrough\metrics\whisper_df_v4gt.csv", index=False)

In [14]:
import pandas as pd
whisper_df_v4gt = pd.read_csv(r".\jupyter_walkthrough\metrics\whisper_df_v4gt.csv")

In [16]:
whisper_df_v4gt = whisper_df_v4gt.dropna()

In [17]:
normalized_ref = werpy.normalize(
    whisper_df_v4gt['gt_transcripts'].values.tolist(),
)
normalized_hypo = werpy.normalize(
    whisper_df_v4gt['whisper_transcripts'].values.tolist(),
)

In [18]:
normalized_ref[0]

'it depends on me if i call you liar or i challenge you cards youre out'

In [19]:
normalized_hypo[0]

'it depends on me if i call you liar or i challenge you to fear go cut'

In [20]:
%%time
summary = werpy.summary(normalized_ref, normalized_hypo)
summary

CPU times: total: 156 ms
Wall time: 200 ms


,wer,ld,m,insertions,deletions,substitutions,inserted_words,deleted_words,substituted_words
0,0.937500,15,16,1,0,3,[to],[],"[(cards, fear), (youre, go), (out, cut)]"
1,4.750000,19,4,1,0,4,[im],[],"[(i, gonna), (gotta, paste), (pace, my), (myse..."
2,0.000000,0,8,0,0,0,[],[],[]
3,1.000000,12,12,0,0,3,[],[],"[(will, with), (seem, seams), (the, be)]"
4,4.750000,19,4,1,0,4,[yes],[],"[(you, i), (got, wish), (solution, you), (yet,..."
...,...,...,...,...,...,...,...,...,...
2605,1.333333,4,3,0,0,1,[],[],"[(that, put)]"
2606,7.000000,7,1,1,0,1,[thank],[],"[(hmm, you)]"
2607,0.000000,0,3,0,0,0,[],[],[]
2608,3.000000,15,5,0,1,3,[],[queens],"[(mhm, please), (oh, one), (hmm, king)]"


In [21]:
I = summary['insertions'].sum()
D = summary['deletions'].sum()
S = summary['substitutions'].sum()
M = summary['m'].sum()

In [22]:
summary.to_csv(r".\jupyter_walkthrough\metrics\v4gt_wer_summary.csv", index=False)

In [23]:
wer = (I + D + S) / M

In [26]:
print(f"wer: {wer}")

wer: 0.5768815753978959


In [27]:
print(f"I, D, S, M: {I, D, S, M}")

I, D, S, M: (3190, 1431, 3933, 14828)


In [28]:
print(f"I/M, D/M, S/M: {I/M, D/M, S/M}")

I/M, D/M, S/M: (0.21513353115727002, 0.09650660911788508, 0.2652414351227408)


# v5

In [29]:
tokenizer = get_tokenizer(multilingual=False)  # use multilingual=True if using multilingual model
number_tokens = [
    i 
    for i in range(tokenizer.eot)
    if all(c in "0123456789" for c in tokenizer.decode([i]).removeprefix(" "))
]

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

whisper_model = whisper.load_model('base.en').to(device)

In [31]:
split_txt_val_path = r'.\preprocessed_data\Ego4D_final_v5\val.txt'
val_uids = []
gt_transcripts_dict = {}
with open(split_txt_val_path) as file:
    for line in file:
        # print(line.split('|')[0])
        val_uids.append(line.split('|')[0])
        gt_transcripts_dict[line.split('|')[0]] = line.split('|')[-1]

In [32]:
len(val_uids)

2715

In [33]:
gt_folder = r'.\raw_data\Ego4D_final_v5\val\Ego4D_final_v5'

In [34]:
%%time
uids = []
whisper_transcripts = []
gt_transcripts = []

for uid in tqdm(val_uids):
    uids.append(uid)
    output = whisper_model.transcribe(
            op.join(gt_folder, f"{uid}.wav"),
            suppress_tokens=[-1] + number_tokens,
    )
    whisper_transcript = output["text"]
    gt_transcript = gt_transcripts_dict[uid]

    whisper_transcripts.append(whisper_transcript)
    gt_transcripts.append(gt_transcript)


  0%|          | 0/2715 [00:00<?, ?it/s]

CPU times: total: 55min 2s
Wall time: 34min 10s


In [35]:
whisper_df_v5gt = pd.DataFrame({
    'uid': uids,
    'whisper_transcripts': whisper_transcripts,
    'gt_transcripts': gt_transcripts,
})

In [36]:
whisper_df_v5gt.head()

,uid,whisper_transcripts,gt_transcripts
0,9d58583c-20de-439d-b1cd-9c2265bdedd8,What's...,what \n
1,4a506707-52ea-493e-98c4-f667e3222d44,"Pretty shit, to be honest.",pretty shit to be honest \n
2,0b67f942-6ebf-403d-aff0-f4e5d62d3140,good morning and alms together,oh when you nail them together they won't be ...
3,c237c6d5-f413-4e25-82e2-af7b408d390a,"Not necessarily, because if they're warped ri...",no not necessary cuz if they're warped right ...
4,22a8a78a-caea-4aa2-be9c-4e02e0d88009,that can reflect this,stand them all down flat \n


In [37]:
whisper_df_v5gt.to_csv(r".\jupyter_walkthrough\metrics\whisper_df_v5gt.csv", index=False)

In [38]:
import pandas as pd
whisper_df_v5gt = pd.read_csv(r".\jupyter_walkthrough\metrics\whisper_df_v5gt.csv")

In [39]:
whisper_df_v5gt = whisper_df_v5gt.dropna()

In [40]:
normalized_ref = werpy.normalize(
    whisper_df_v5gt['gt_transcripts'].values.tolist(),
)
normalized_hypo = werpy.normalize(
    whisper_df_v5gt['whisper_transcripts'].values.tolist(),
)

In [41]:
normalized_ref[0]

'what'

In [42]:
normalized_hypo[0]

'whats'

In [43]:
%%time
summary = werpy.summary(normalized_ref, normalized_hypo)
summary

CPU times: total: 141 ms
Wall time: 189 ms


,wer,ld,m,insertions,deletions,substitutions,inserted_words,deleted_words,substituted_words
0,4.000000,4,1,0,0,1,[],[],"[(what, whats)]"
1,0.000000,0,5,0,0,0,[],[],[]
2,3.083333,37,12,0,7,4,[],"[oh, they, wont, be, as, hard, right]","[(when, good), (you, morning), (nail, and), (t..."
3,0.611111,11,18,0,1,2,[],[no],"[(necessary, necessarily), (cuz, because)]"
4,3.800000,19,5,0,1,4,[],[stand],"[(them, that), (all, can), (down, reflect), (f..."
...,...,...,...,...,...,...,...,...,...
2665,0.166667,7,42,1,0,1,[player],[],"[(a, the)]"
2666,2.960784,151,51,0,49,1,[],"[id, read, the, instructions, to, you, right, ...","[(first, like)]"
2667,0.339286,19,56,0,5,1,[],"[a, and, so, on, and]","[(then, when)]"
2668,0.750000,30,40,10,0,0,"[dukkid, dont, accept, ahh, dukk, nikr, nikr, ...",[],[]


In [44]:
I = summary['insertions'].sum()
D = summary['deletions'].sum()
S = summary['substitutions'].sum()
M = summary['m'].sum()

In [45]:
summary.to_csv(r".\jupyter_walkthrough\metrics\v5gt_wer_summary.csv", index=False)

In [46]:
wer = (I + D + S) / M

In [47]:
print(f"wer: {wer}")

wer: 0.6610960757780785


In [48]:
print(f"I, D, S, M: {I, D, S, M}")

I, D, S, M: (2875, 2097, 4799, 14780)


In [49]:
print(f"I/M, D/M, S/M: {I/M, D/M, S/M}")

I/M, D/M, S/M: (0.19451962110960758, 0.1418809201623816, 0.3246955345060893)
